In [ ]:
# 🧹 Data Loading for Twitter Fake vs Verified News Detection

import pandas as pd
import numpy as np
import re
import string

# Load PolitiFact fake and real datasets
fake_df = pd.read_csv("data/politifact_fake.csv")
real_df = pd.read_csv("data/politifact_real.csv")


print("Fake news samples:", fake_df.shape)
print("Real news samples:", real_df.shape)

# Add labels
fake_df["label"] = 0   # Fake
real_df["label"] = 1   # Real

# Combine into a single dataframe
data = pd.concat([fake_df, real_df], ignore_index=True)
print("Total combined dataset:", data.shape)

# Keep only the title + label
data = data[["title", "label"]]

data.head()
